In [ ]:
#Here I have done some data exploration and have build model on Random Forest Regression to predict the sales

In [ ]:
import pandas as pd



In [ ]:
df=pd.read_csv('../input/vehicle-dataset-from-cardekho/car data.csv')

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
print(df['Seller_Type'].unique())
print(df['Transmission'].unique())
print(df['Owner'].unique())
print(df['Fuel_Type'].unique())

In [ ]:
#check missing null values

df.isnull().sum()

In [ ]:
df.isna().sum()

In [ ]:
df.describe()

In [ ]:
df.columns

In [ ]:
final_dataset= df[[ 'Year', 'Selling_Price', 'Present_Price', 'Kms_Driven','Fuel_Type', 'Seller_Type', 'Transmission', 'Owner']]

In [ ]:
final_dataset.head()

In [ ]:
final_dataset['Current_Year']=2020

In [ ]:
final_dataset.head()

In [ ]:
final_dataset['no_years']= final_dataset['Current_Year']-final_dataset['Year']

In [ ]:
final_dataset.head()

In [ ]:
final_dataset.drop('Year', axis=1, inplace=True)

In [ ]:
final_dataset.drop('Current_Year', axis=1, inplace=True)

In [ ]:
final_dataset.head()

In [ ]:
#Convert category in one hot lable

In [ ]:
final_dataset=pd.get_dummies(final_dataset,drop_first=True)

In [ ]:
final_dataset.head()

In [ ]:
final_dataset.corr()

In [ ]:
import seaborn as sns

In [ ]:
sns.pairplot(final_dataset)

In [ ]:

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
corrmat=final_dataset.corr()
top_corr_feature=corrmat.index
plt.figure(figsize=(20,20))
g= sns.heatmap(final_dataset[top_corr_feature].corr(),annot=True,cmap='RdYlGn')

In [ ]:
#independent and dependent feature

x=final_dataset.iloc[:,1:]
y=final_dataset.iloc[:,0]

In [ ]:
y.head()

In [ ]:
###Feature Importance

from sklearn.ensemble import ExtraTreesRegressor
model=ExtraTreesRegressor()
model.fit(x,y)

In [ ]:
print(model.feature_importances_)

In [ ]:
#plot graph of feature importance 

feat_importances= pd.Series(model.feature_importances_, index=x.columns)
feat_importances.nlargest(5).plot(kind='barh')
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3)

In [ ]:
x_train.shape

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf_random=RandomForestRegressor()

In [ ]:
###Hyperparameter Tuning - Randomize search CV
import numpy as np
n_estimators=[int(x) for x in np.linspace(start=100, stop=1500, num=15)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(5, 50, num = 10)]
# max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10, 15, 100]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 5, 10]

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}

print(random_grid)

In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()


In [ ]:
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid,scoring='neg_mean_squared_error', n_iter = 15, cv = 5, verbose=2, random_state=42, n_jobs = 1)

In [ ]:
rf_random.fit(x_train,y_train)

In [ ]:
rf_random.best_params_

In [ ]:
rf_random.best_score_

In [ ]:
predictions=rf_random.predict(x_test)

In [ ]:
sns.distplot(y_test-predictions)

In [ ]:
plt.scatter(y_test,predictions)

In [ ]:
from sklearn import metrics
print('MAE:', metrics.mean_absolute_error(y_test, predictions))
print('MSE:', metrics.mean_squared_error(y_test, predictions))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, predictions)))


In [ ]:
 from sklearn.ensemble import AdaBoostRegressor


In [ ]:
ada=AdaBoostRegressor()

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
#Creating a grid of hyperparameters
grid_params = {  'n_estimators': [100,200,300],
                'learning_rate': [0.02,0.5,1],
                 'loss': ['linear', 'square', 'exponential']}

In [ ]:
#Building a 3 fold CV GridSearchCV 
RCV = RandomizedSearchCV(estimator = ada, param_distributions = grid_params,scoring='neg_mean_squared_error', n_iter = 15, cv = 5, verbose=2, random_state=42, n_jobs = 1)



In [ ]:
#Fitting the grid to the training 
RCV.fit(x_train, y_train)


In [ ]:
RCV.best_params_

In [ ]:
predict=RCV.predict(x_test)

In [ ]:
plt.scatter(y_test,predict)

In [ ]:
sns.distplot(y_test-predict)

In [ ]:
from sklearn import metrics
print('MAE:', metrics.mean_absolute_error(y_test, predict))
print('MSE:', metrics.mean_squared_error(y_test, predictions))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, predictions)))

In [ ]:
#Here we have compared two models 1) Random Forest Regressor 2) AdaBoost
# We can see Random Forest Preformance better tha AdaBoost    